In [4]:
#Bu kod, uygulamanın asıl fonksiyonlarını ve arayüzünü içerir.

import seaborn as sns
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import pandas as pd
import numpy as np

#veri setleri yüklenir ve np.array() ve pd.DataFrame() türlerindeki verilere çevrilir.
UserSongs=pd.read_excel("UsersAndSongs.xlsx")
UserNumericDf=pd.read_excel("UsersLogical.xlsx")
UserNumericNp=[]

for i in np.array(UserNumericDf):
    i=np.delete(i,0)
    UserNumericNp.append(i)

UserNumericDf=pd.DataFrame(UserNumericNp,columns=["age","musical_konwledge","acoustic","classical","jazz","pop","rock","hip-hop","world-music"])
UserNumericNp=np.array(UserNumericNp)

#Verilerin Standardizasyonu yapılır.
scaler=preprocessing.StandardScaler().fit(UserNumericNp)

UserNumericNpScaled=scaler.transform(UserNumericNp)
UserNumericDfScaled=pd.DataFrame(list(UserNumericNpScaled))

#7'li gruplar halinde kullanıcılar kümelenir (k saıyısının seçimi ve görselleştirilmesi için lütfen Ek'i ve sunumu inceleyiniz.)
KM1=KMeans(n_clusters=7)
Clusters=KM1.fit_predict(UserNumericDfScaled)
UserNumericDfScaled_Predicted=UserNumericDfScaled
UserNumericDfScaled_Predicted["Label"]=Clusters
UserNumericDf_Predicted=UserNumericDf
UserNumericDf_Predicted["Label"]=Clusters

y=UserNumericDfScaled_Predicted["Label"].values
x=UserNumericDfScaled_Predicted.drop(["Label"],axis=1)
# veri setinin kullanıcı etikletlerini içeren bir kopyası oluşturulur.

def Nearest10Users(UserIndex):
    classifier=DecisionTreeClassifier()
    model=classifier.fit(x,y)
    UserClass=int(model.predict(x.iloc[[UserIndex]])[0])
    #Kullanıcının sınıfı belirlenir.
    #Veri setinin etikeli kopyası kullanılmamıştır, bu ileride yeni katılan etiketsiz kullanıcıların sınıflandırılmasının 
    #simüle edilmesi içindir.
    
    OurUser_Songs=UserSongs.iloc[UserIndex]
    OurUser_Info=UserNumericDf.iloc[UserIndex]
    
    SameClassUsers=pd.concat([UserNumericDf,UserSongs],axis=1)
    SameClassUsers=SameClassUsers[SameClassUsers["Label"]==UserClass]
    
    OurUser_SongList=list(OurUser_Songs)[2:]
    OurUserAge=int(pd.concat([UserNumericDf,UserSongs],axis=1).iloc[UserIndex]["age"])
    OurUserMusicalK=int(pd.concat([UserNumericDf,UserSongs],axis=1).iloc[UserIndex]["musical_konwledge"])
    
    OtherUsers=SameClassUsers.drop(UserIndex)
    OtherUserIndex=list(OtherUsers.index)
    
    #hedef kullanıcının ve aynı sınıftaki diğer kullanıcıların özellikleri alınır.
    
    CommonSongCount=[]
    MusicalKDifference=[]
    AgeDifference=[]
    UserNames=[]
    
    for i in OtherUserIndex:
        User=UserSongs.iloc[i]
        UserSongList=list(User)[2:]
    
        CommonSongs = set(OurUser_SongList).intersection(set(UserSongList))
        CNum = len(CommonSongs)
        CommonSongCount.append(CNum)
    
        UserAge=int(pd.concat([UserNumericDf,UserSongs],axis=1).iloc[i]["age"])
        UserMusicalK=int(pd.concat([UserNumericDf,UserSongs],axis=1).iloc[i]["musical_konwledge"])
    
        AgeDifference.append(abs(UserAge-OurUserAge))
        MusicalKDifference.append(abs(UserMusicalK-OurUserMusicalK))
    
        UserNames.append(UserSongs.iloc[i]["Username"])
    
    Difference_Df=pd.DataFrame(list(zip(OtherUserIndex,UserNames,CommonSongCount,MusicalKDifference,AgeDifference)),
                           columns=["total index","username","Common songs","Musical Knowledge difference","age difference"])
    
    
    features=np.array(list(zip(CommonSongCount,MusicalKDifference,AgeDifference)))
    model=NearestNeighbors(n_neighbors=10,algorithm="auto")
    model.fit(features)
    target=np.array([[30,0,0]])
    uzaklıklar,indisler=model.kneighbors(target)
    
    # NearestNeighbors algoritmasına uygun şekilde yaş farkı, müzik bilgisi farkı ve ortak şarkı sayısı features adlı
    #array'e kaydedildikten sonra En yakın komşu bulmak için model eğitilir. En yakın 10 kullanıcının indexleri ve uzaklık değerleri
    #alınır
    
    def EnUzakMesafe():
        trying=list(features)
        trying.append([0,10,48])
        trying=np.array(trying)
        model=NearestNeighbors(n_neighbors=len(trying),algorithm="auto")
        model.fit(trying)
        u,ind=model.kneighbors(target)
        en_uzak=u[0][len(trying)-1]
        return en_uzak
        #bu, daha sonra benzerlik yüzdesini hesaplamak için ihtiyaç duyulacak en uzak mesafe değerini bulmak için oluşturulan
        #bir fonksiyondur.
    
    uzaklık_yuzde=((EnUzakMesafe()-uzaklıklar)/EnUzakMesafe())*100
    
    Benzerlik_siralamasi=[]
    Kullanici_adlari=[]
    Benzerlik_Yuzdeleri=[]
    Ayni_sarkilar=[]
    
    for i in list(indisler[0]):
        total_index=Difference_Df.iloc[i]["total index"]
        nameoftheuser=Difference_Df.iloc[i]["username"]
    
        User=UserSongs.iloc[total_index]
        UserSongList=list(User)[2:]
    
        CommonSongs = list(set(OurUser_SongList).intersection(set(UserSongList)))
        
        Benzerlik_siralamasi.append(list(indisler[0]).index(i)+1)
        Kullanici_adlari.append(nameoftheuser)
        Yuzdelik=uzaklık_yuzde[0][list(indisler[0]).index(i)]
        Benzerlik_Yuzdeleri.append(round(float(Yuzdelik),1))
        Ayni_sarkilar.append(CommonSongs)
        
    output=[Benzerlik_siralamasi,Kullanici_adlari,Benzerlik_Yuzdeleri,Ayni_sarkilar]
    #en yakın 10 kullanıcının erişilen özellikleri derlenir ve benzerlik sıralaması, benzerlik yüzdesi, kullanıcı adları ve
    #ortak şarkıları içeren listeler oluşturulup fonksiyonun çıktısı olarak döndürülür.
    
    
    return output

In [5]:
#ARAYÜZÜN OLUŞTURULMASI
import tkinter as tk
from PIL import Image, ImageTk
import time

form=tk.Tk()
form.title("Fizy - Müzik İkizini Bul")
form.geometry("350x700+750+70")
form.state("iconic")
form.configure(bg="#ffd900")

giris_bilgi="""
Fizy'ye hoşgeldiniz. 'Müzik İkizini Bul' özelliğiyle 
sizinle aynı şarkıları dinleyen, Aynı müziklere ilgi
duyan aynı yaş grubundaki kullanıcıları bulabilir,
onlarla iletişime geçebilir, onlara şarkı önerebilir 
veya onların önerilerine göz atabilirsiniz. Yeni Müzik
arkadaşları edinmek için şimdi deneyin!
"""
Label1=tk.Label(form,text=giris_bilgi,bg="#ffd900",fg="#ba9e00")
Label1.place(x=35,y=120)

GirisLabel=tk.Label(form,text="Kullanıcı Adınız:",font=("Arial",12),bg="#ffd900")
GirisLabel.place(x=10,y=250)

Ad_Giris=tk.Entry()
Ad_Giris.place(x=125,y=252)

Bekle=tk.Label(form,bg="#ffd900")
Bekle.place(x=90,y=320)

#en yakın 10 kullanıcının ekranda gösterilmesi
def islem():
    FizyKullaniciAdi=Ad_Giris.get()
    FizyKullaniciIndex=int(UserSongs[UserSongs["Username"]==FizyKullaniciAdi].index[0])
    sonuc=Nearest10Users(FizyKullaniciIndex)
    
    siralama=sonuc[0]
    kullaniciAdlari=sonuc[1]
    yuzdeler=sonuc[2]
    ortaksarkilar=sonuc[3]
    zippedList=list(zip(siralama,kullaniciAdlari,yuzdeler,ortaksarkilar))
    
    UstPencere=tk.Toplevel(bg="black")
    UstPencere.wm_attributes("-topmost",True)
    UstPencere.geometry("1200x550")
    UstPencere.title("İşte Müzik İkizleriniz!")

    for i in zippedList:
        sira=i[0]
        ad=i[1]
        yuzde=i[2]
        
        if len(i[3])>1:
            if len (i[3])>2:
                sarkilar=i[3][0]+", "+i[3][1]+" ve daha fazlası"
            else:
                sarkilar=i[3][0]+" ve "+i[3][1]
        else:
            sarkilar=i[3][0]
            
        sira_ad=str(sira)+". "+ad
        benzeme="Size %{} oranında benziyor!".format(str(yuzde))
        ortaklar="Ortak Dinlediğiniz Şarkılar: "+sarkilar
        
        isim=tk.Label(UstPencere,text=sira_ad,bg="#ffd900",fg="black")
        isim.place(x=30,y=30+(50*(zippedList.index(i))))
        
        bilgi=tk.Label(UstPencere,text=benzeme+"\n"+ortaklar,bg="#3d3d3d",fg="white")
        bilgi.place(x=160,y=20+(50*(zippedList.index(i))))
        
        mesaj=tk.Button(UstPencere,text="İletişime Geç!",bg="white")
        mesaj.place(x=1100,y=20+(50*(zippedList.index(i))))
        
    UstPencere.mainloop()
    
    
def changeTextAndContinue():
    Bekle.config(text="Müzik İkizleriniz bulunuyor...\nLütfen Bekleyiniz.")
    islem()
    
Dene=tk.Button(form,text="Müzik İkizini Bul!",bg="black",fg="#ffd900",command=changeTextAndContinue)
Dene.place(x=130,y=285)

dizin="C://Users//asus//Desktop//zeynep calisma//kodlama//TURKCELL YAPAY ZEKA//Hackaton_Proje//Project_clear//Images//Fizy.png"
fizy_logo=ImageTk.PhotoImage(Image.open(dizin))
logoEtiket=tk.Label(form,image=fizy_logo,bg="#ffd900")
logoEtiket.place(x=125,y=20)

dizin2="C://Users//asus//Desktop//zeynep calisma//kodlama//TURKCELL YAPAY ZEKA//Hackaton_Proje//Project_clear//Images//Turkcell.png"
turkcell=ImageTk.PhotoImage(Image.open(dizin2))
resimEtiket=tk.Label(form,image=turkcell,bg="#ffd900")
resimEtiket.place(x=-120,y=320)


form.mainloop()

NOT: Arayüzü çalıştırırken UsersAndSongs.xlsx dosyasında bulunan kullanıcı adlarını kullanınız. "Müzik İkizini Bul!" butonuna bastıktan sonra Arayüzün penceresinde "Yanıt Vermiyor" yazısının çıkması normaldir, Müzik ikizlerinin ekranına ulaşmadan önce biraz beklenmesi gerekmektedir.